## Binomial Option Pricing Model

We can implement the Binomial Option Pricing Model in Python as follows. 

<br>
<br>

Let's start with specifing payoff functions for calls and puts. 

<br>

Recall that the payoff for call option is:

$$
\mbox{Call Payoff} = \max{\{S_{t} - K, 0.0 \}}
$$

The put payoff is:

$$
\mbox{Put Payoff} = \max{\{K - S_{t}, 0.0\}}
$$

<br>

We can implement these in code as follows.

In [2]:
## First import numpy
import numpy as np

## Call Payoff Function
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)


Let's check these with $S = \$100$ and $K_{1} = \$95$ and $K_{2} = \$105$

In [3]:
spot = 100.0
strikes = np.array([95.0, 105.0])
strikes

array([ 95., 105.])

In [4]:
callPO = callPayoff(spot, strikes)
callPO

array([5., 0.])

In [5]:
putPO = putPayoff(spot, strikes)
putPO

array([0., 5.])

They seem to be working just fine!

<br>

- $r = 8\%$
- $\sigma = 30\%$
- $T = 1$ year
- $\delta = 0.0$ (no dividends)
- $n = 1$ (a single period)
- $h = T/n = 1$

Also recall that the formula for the parameters $u$ and $d$ are given by:

$$
\begin{align}
u &= e^{\{(r - \delta)h + \sigma \sqrt{h}\}} \\
d &= e^{\{(r - \delta)h - \sigma \sqrt{h}\}} 
\end{align}
$$

<br>

These can be implemented in Python as follows:

In [24]:
r = 0.08       ## time step 
v = 0.30       ## volatility
q = 0.0        ## dividend
expiry = 1.0
n = 1.0        ## periods
h = expiry / n 
S = 100.0      ## spot price
K = 95.0       ## strike price

u = np.exp((r - q) * h + v * np.sqrt(h))
d = np.exp((r - q) * h - v * np.sqrt(h))

In [25]:
print((u, d))

(1.4622845894342245, 0.8025187979624785)


### Implementing the Single Period Model

Now we can implement the single period model in two ways. Recall that we could write the model in these two forms:

__No-Arbitrage Form:__

$$
f_{0} = \Delta S + B
$$

with 

$$
\Delta = \frac{f_{u} - f_{d}}{S(u - d)}
$$

and

$$
B = e^{-r h}\left[\frac{u f_{d} - d f_{u}}{(u - d)}\right]
$$

Where $f$ stands for either a call or put. Let's see this in action for a call option.

In [26]:
fu = callPayoff(u * S, K)
fd = callPayoff(d * S, K)
D = (fu - fd)/(S * (u - d))                        ## Delta
B = np.exp(-r * h) * ((u * fd - d * fu)/ (u - d))  ## B 
f = S * D + B
f

20.124540120626122

__Risk-Neutral Form__

We saw that we could also put the single period model in risk-neutral form. There we only needed to define one additional parameter $p^{\ast}$, which we do as follows:

$$
p^{\ast} = \frac{e^{(r - \delta)h} - d}{u - d}
$$

<br>

Then the single period model can also be written as:

$$
f_{0} = e^{-rh} \left[(p^{\ast}) f_{u} - (1 - p^{\ast}) f_{d} \right]
$$

<br>

Which we implement in Python as follows:

In [27]:
pstar = (np.exp((r - q) * h) - d) / (u - d)
pstar

0.4255574831883412

In [28]:
f = np.exp(-r * h) * (pstar * fu + (1 - pstar) * fd)
f

20.124540120626122

From this we can see that they are the exact same solution! Yeet!!!

<br>

But we're now experienced Python programmers. Instead of inline code, let's see if we can abstract this away into a function.

In [34]:
def singlePeriodBinomialModel(S, K, r, v, q, T, n, payoff):
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    fu = payoff(u * S, K)
    fd = payoff(d * S, K)
    f = np.exp(-r * h) * (pstar * fu + (1 - pstar) * fd)
    
    return f
    

In [35]:
callPrc = singlePeriodBinomialModel(S, K, r, v, q, expiry, n, callPayoff)
callPrc

20.124540120626122

Yeet!!!

<br>

Also, note that we can now also price the put for free! This is actually an example of polymorphism (which we'll return to). It is possible because functions are first-class objects in Python so we can pass functions to functions. 

In [37]:
putPrc = singlePeriodBinomialModel(S, K, r, v, q, expiry, n, putPayoff)
putPrc

7.82059302735651

### The Multi-Period Binomial Model

Well, great but the single period model is pretty simplistic. We don't think it is sufficient for pricing actual real-world options. It's just a toy model. Recall that we used backward induction to solve recursively for the option price in a multi-period model applying at each set of nodes the single period model. So we can build on the single period model to solve for the option price in a more complex model. How can we implement this in Python code? Let's start by recalling our simplification for European options. 

<br>

Let's start with a two-period model ($n = 2$). The terminal prices on the last nodes of the tree are given as follows:

- $uuS$
- $udS = duS$
- $ddS$

Using the way of thinking embedded in the risk-neutral form of the single period model we can write the option premium as:

$$
f_{0} = e^{-r T} \left[p_{uu} f_{uu} + p_{ud} f_{ud} + p_{dd} f_{dd} \right]
$$

Assuming we could know the risk-neutral probabilities $p_{uu}, p_{ud}, p_{dd}$